In [1]:
base_dir = './'
fig_dir = './figures/'
model_dir = './models/'

In [2]:
import os
import sys

sys.path.insert(0, os.getcwd() + '\\src')

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import trange

---

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [5]:
print('CUDA available :', torch.cuda.is_available())
# device = torch.device('cpu')
device = torch.device('cuda')

CUDA available : True


In [6]:
n_timestamps = 5
lag = 1

data_dir = f'./data/n_timestamps_{n_timestamps}__lag_{lag}/' + 'data_split.pt'

data_split = torch.load(data_dir)
train_data, train_pos_enc, train_dates = data_split['train'].values()
valid_data, valid_pos_enc, valid_dates = data_split['valid'].values()
test_data, test_pos_enc, test_dates = data_split['test'].values()

# Concat Date Positional Encoding
X_train = torch.cat((train_data, train_pos_enc), dim=-1)
X_valid = torch.cat((valid_data, valid_pos_enc), dim=-1)
X_test = torch.cat((test_data, test_pos_enc), dim=-1)

# Get currency names
currency_names = pd.read_csv(f'./data/n_timestamps_{n_timestamps}__lag_{lag}/cleansed_data.csv').columns.tolist()

del data_split

In [7]:
N = X_train.shape[0] + X_valid.shape[0] + X_test.shape[0]
_, T, D = X_train.shape
print(f'Data shape : (*, {T}, {D})')
print(f'    - Train : {train_dates[0]} ~ {train_dates[-1]} ({X_train.__len__():>4d}, {X_train.__len__()/N*100:.2f}%)')
print(f'    - Valid : {valid_dates[0]} ~ {valid_dates[-1]} ({X_valid.__len__():>4d}, {X_valid.__len__()/N*100:.2f}%)')
print(f'    - Test  : {test_dates[0]} ~ {test_dates[-1]} ({X_test.__len__():>4d}, {X_test.__len__()/N*100:.2f}%)')

Data shape : (*, 5, 36)
    - Train : 2002-01-08 ~ 2015-05-07 (3412, 63.99%)
    - Valid : 2015-05-08 ~ 2018-09-03 ( 853, 16.00%)
    - Test  : 2018-09-04 ~ 2022-10-31 (1067, 20.01%)


---

In [8]:
from encoder import Encoder
from itertools import product

In [9]:
manifold_names = {'euclidean':'Euclidean', 
                  'p_plane':'Poincare-Halfplane', 
                  'sphere':'Hypersphere'}

- Training

In [10]:
dim_data = (T, D)
layer_configs = {}

candidates = list(product(['sphere', 'p_plane', 'euclidean'], 
                          list(range(20, 40, 4))))